In [ ]:
import pandas as pd
import numpy as np
df = pd.read_json('..\data\\raw_sources\data_bike_exchange.json', lines=True)
df.head(10)

In [ ]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('all')

In [ ]:
def print_nans_perc(df):
    n = len(df)
    nans_perc = ((df.isna().sum() / n) * 100).sort_values(ascending=True)
    print('len - ', n)
    for index, val in zip(nans_perc.index, nans_perc):
        print(index + ': ', val)
print_nans_perc(df)
df['Type'].unique()

In [ ]:
bike_types = df['Type'].unique()
for Btype in bike_types:
    print('\n\n'+ Btype + ':\n')
    print_nans_perc(df[df['Type'] == Btype])

In [ ]:
usefull_cols = ['ID','Title','Type','Price now','Brand','Item condition', 'Material', 'Rear Derailleur', 'Groupset', 'Braking Type', 'Wheel Size']
usefull_cols_mtb = ['ID','Title','Price now','Brand','Item condition', 'Material', 'Wheel Size', 'Rear Derailleur', 'Riding Style']

In [ ]:
full_orig = df[df['Type'] == 'Full suspension_num Mountain Bikes'][usefull_cols_mtb]
hardtrail_orig = df[df['Type'] == 'Hardtail Mountain Bikes'][usefull_cols_mtb]
full_orig['suspension_num'] = 1
hardtrail_orig['suspension_num'] = 0.5
mtb_orig = pd.concat([full_orig, hardtrail_orig], axis = 0)
print_nans_perc(mtb_orig)

In [ ]:
mtb = mtb_orig.copy()
mtb.rename(columns={'Price now': 'Price'}, inplace=True)
mtb['Price'].replace('[A-Za-z, \$]','',inplace=True, regex=True)
mtb = mtb[mtb['Price'] != '']
mtb['Price'] = mtb['Price'].astype(float)
mtb

In [ ]:
mtb.rename(columns={'Item condition': 'isNew', 'Rear Derailleur': 'Groupset'}, inplace=True)
mtb['isNew'].replace({'Used': 0, 'New': 1},inplace=True, regex=True)
mtb['isNew'] = mtb['isNew'].astype(int)
mtb

In [ ]:
material_dict = {'Chromoly': 0.15, 'Steel': 0.3, 'Aluminum': 0.45, 'Titanium': 0.6, 'Carbon Alloy Composite': 0.75, 'Carbon': 0.9}
mtb['material_num'] = mtb['Material'].replace(material_dict, regex=True)
mtb['material_num'] = mtb['material_num'].astype(float)
mtb

In [ ]:
print(mtb.groupby('Groupset').size())
#mtb['Rear Derailleur'].unique()

In [ ]:
mtb = mtb[mtb['Groupset'] != 'N/A']
mtb = mtb[mtb['Groupset'] != 'Singlespeed']
mtb['e_shift_num'] = mtb['Groupset'].apply(lambda x: 1 if any(ele in x for ele in ['Di2', 'AXS', 'eTap']) else 0)

RD_dict = {    
        'SRAM XX1 Eagle': 0.9, 'SRAM XX1': 0.9, 'SRAM X01 Eagle': 0.8, 'SRAM X01': 0.8, 'SRAM GX Eagle': 0.7, 'SRAM GX': 0.7, 'SRAM NX Eagle': 0.6, 
        'SRAM NX': 0.6, 'SRAM X9': 0.6, 'SRAM X7': 0.5, 'SRAM X5': 0.4,'SRAM X1': 0.8,'SRAM SX Eagle': 0.5, 'SRAM SX': 0.5,
        'SRAM XX': 1, 'SRAM X0': 0.95,

        'Shimano XTR': 0.9, 'Shimano Deore XT': 0.8, 'Shimano XT': 0.7, 'Shimano Saint': 0.7, 'Shimano Zee': 0.6, 'Shimano SLX': 0.6, 'Shimano Deore': 0.5, 
        'Shimano Alivio': 0.4, 'Shimano Acera': 0.3, 'Shimano Altus': 0.2, 'Shimano Tourney': 0.1, 'Shimano TX800': 0.1, 'Shimano TX35': 0.1, 'Shimano TY500': 0.1, 'Shimano M310': 0.1,
        'Shimano XTR Di2': 1,
        }

mtb['groupset_num'] = mtb['Groupset'].replace(RD_dict, regex=True, inplace=False)
mtb

In [ ]:
mtb['Wheel Size'].unique()
#mtb.groupby('Wheel Size').size()

In [ ]:
mtb = mtb[mtb['Wheel Size'] != '24"']

In [ ]:
wheel_size_dict = {'27.5"': 0.66, '29"': 1, '650b': 0.66, '26"': 0.33}
mtb['wheel_num'] = mtb['Wheel Size'].replace(wheel_size_dict, regex=True)
mtb

In [ ]:
import re

def find_year(x):
    four_num = re.findall(r'[0-9]{4}', x)
    four_num = [int(num) for num in four_num]
    for num in four_num:
        if 1990 <= num <= 2024:
            return num
    return np.nan

mtb['Production_year'] = mtb['Title'].apply(find_year)
mtb['Production_year'].fillna(mtb['Production_year'].median(), inplace=True)

In [ ]:
mtb['brake_num'] = 1

In [ ]:
num_cols = ['ID', 'Price','Production_year', 'isNew', 'suspension_num','material_num', 'groupset_num', 'e_shift_num', 'wheel_num', 'brake_num']

In [ ]:
mtb_num = mtb[num_cols]
mtb_num.to_json('..\data\exch_mtb.json', lines=True, orient='records')
mtb_num
